In [2]:
import os
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ===========================
# Configuration
# ===========================
base_path = r'E:\SAND_Challenge_task1_dataset\task1\training'

plot_dir = "plots"
os.makedirs(plot_dir, exist_ok=True)

# ===========================
# Helper Functions
# ===========================

def analyze_audio(file_path, label):
    """Extract MFCC and audio quality features for a single file"""
    try:
        y, sr = librosa.load(file_path, sr=None)
        
        # Duration and energy (audio quality)
        duration = librosa.get_duration(y=y, sr=sr)
        energy = np.mean(y ** 2)
        
        # Compute MFCCs
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_var = np.var(mfcc, axis=1)
        
        # Return as dict
        data = {
            "file": os.path.basename(file_path),
            "label": label,
            "duration": duration,
            "energy": energy
        }
        for i in range(13):
            data[f"mfcc{i+1}_mean"] = mfcc_mean[i]
            data[f"mfcc{i+1}_var"] = mfcc_var[i]
            
        return data
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


def plot_mfcc(y, sr, file_name):
    """Save MFCC spectrogram plot"""
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mfcc, x_axis='time', sr=sr)
    plt.colorbar()
    plt.title(f"MFCC - {file_name}")
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, f"{file_name}_mfcc.pdf"))
    plt.close()


def plot_waveform(y, sr, file_name):
    """Save waveform plot"""
    plt.figure(figsize=(10, 3))
    librosa.display.waveshow(y, sr=sr)
    plt.title(f"Waveform - {file_name}")
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, f"{file_name}_waveform.pdf"))
    plt.close()

# ===========================
# Main Analysis
# ===========================

summary = []

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder}")
        for file in os.listdir(folder_path):
            if file.endswith(".wav") or file.endswith(".mp3"):
                file_path = os.path.join(folder_path, file)
                result = analyze_audio(file_path, folder)
                if result:
                    summary.append(result)
                
                # Plot MFCC and waveform for first 2 samples per class
                if len([s for s in summary if s["label"] == folder]) <= 2:
                    y, sr = librosa.load(file_path, sr=None)
                    plot_mfcc(y, sr, f"{folder}_{os.path.splitext(file)[0]}")
                    plot_waveform(y, sr, f"{folder}_{os.path.splitext(file)[0]}")

# ===========================
# Save Summary & Plots
# ===========================

df = pd.DataFrame(summary)
df.to_csv("mfcc_audio_quality_summary.csv", index=False)
print("✅ Analysis complete! Results saved in mfcc_audio_quality_summary.csv and plots/ folder.")

# ===========================
# Example Dataset-Level Plots
# ===========================

# Duration distribution
plt.figure(figsize=(8, 4))
df.boxplot(column='duration', by='label', grid=False)
plt.title("Audio Duration by Category")
plt.suptitle("")
plt.ylabel("Duration (seconds)")
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, "duration_boxplot.pdf"))
plt.close()

# MFCC1 Mean Distribution
plt.figure(figsize=(8, 4))
for label in df['label'].unique():
    plt.hist(df[df['label'] == label]['mfcc1_mean'], bins=30, alpha=0.5, label=label)
plt.title("MFCC1 Mean Distribution Across Classes")
plt.xlabel("MFCC1 Mean")
plt.ylabel("Count")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, "mfcc1_distribution.pdf"))
plt.close()

print("📊 All plots saved successfully in 'plots/' directory.")


Processing folder: phonationA
Processing folder: phonationE
Processing folder: phonationI
Processing folder: phonationO
Processing folder: phonationU
Processing folder: rhythmKA
Processing folder: rhythmPA
Processing folder: rhythmTA
✅ Analysis complete! Results saved in mfcc_audio_quality_summary.csv and plots/ folder.
📊 All plots saved successfully in 'plots/' directory.


<Figure size 800x400 with 0 Axes>